# AML Task 2 : Disease Classification

Useful links :
* [combining models in Keras](https://machinelearningmastery.com/model-averaging-ensemble-for-deep-learning-neural-networks/#:~:text=The%20simplest%20way%20to%20develop,each%20of%20the%20trained%20models.)

* [technique to handle imbalanced data](https://www.kdnuggets.com/2017/06/7-techniques-handle-imbalanced-data.html)

* [TensorFlow MultiClass Classification with MLP](https://medium.com/@canerkilinc/hands-on-tensorflow-2-0-multi-label-classifications-with-mlp-88fc97d6a7e6)

## Install

In [ ]:
import sys
!{sys.executable} -m pip install tensorflow
import sys
!{sys.executable} -m pip install pandas

## Imports

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from sklearn.utils import shuffle
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split
from random import randrange

print('Imports : done')

DEBUG:matplotlib:$HOME=/home/atroska
DEBUG:matplotlib:CONFIGDIR=/home/atroska/.config/matplotlib
DEBUG:matplotlib:matplotlib data path: /usr/local/lib/python3.6/dist-packages/matplotlib/mpl-data
DEBUG:matplotlib:loaded rc file /usr/local/lib/python3.6/dist-packages/matplotlib/mpl-data/matplotlibrc
DEBUG:matplotlib:matplotlib version 3.0.2
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'zipimport', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_weakrefset', '_bootlocale', '_locale', 'site', 'os', 'errno', 'stat', '_stat', 'posixpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltins', 'sysconfig', '_sysconfigdata_m_linux_x86_64-linux-gnu', 'types', 'functools', '_functools', 'collections', 'op

DEBUG:tensorflow:Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
DEBUG:matplotlib:CACHEDIR=/home/atroska/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/atroska/.cache/matplotlib/fontlist-v300.json
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


Imports : done


## Read data

In [2]:
X_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv")
X_test = pd.read_csv("X_test.csv")
sample=pd.read_csv("sample.csv")

In [3]:
X_train_copy=X_train.to_numpy()[:,1:]
y_train_copy=y_train.to_numpy()[:,1:]
X_test_copy=X_test.to_numpy()[:,1:]

In [ ]:
print('Data read and copy')

In [ ]:
y_train.describe()

In [ ]:
X_test.describe()

In [ ]:
y_train_copy

## Pre-processing DATA

### Under-sampling 

In [ ]:
#Take randomly 600 sample from the sample with classification 
indexes = np.where(y_train_copy==1)[0]
count = len(indexes)
for i in range(3000):
    random = randrange(count)
    index = indexes[random]
    X_train_copy = np.delete(X_train_copy, index, 0)
    y_train_copy = np.delete(y_train_copy, index, 0)
    indexes = np.where(y_train_copy==1)[0]
    count = len(indexes)
print('Under-sampling done')

In [ ]:
len(X_train_copy)==len(y_train_copy)

In [ ]:
len(X_train_copy)

## Ensemble different resampled datasets

In [4]:
def create6balancedsets(x, y):
    
    #TO BE CONTINUED
    x_0 = x[np.where(y == 0)[0]]
    y_0 = y[np.where(y == 0)[0]]
    x_2 = x[np.where(y == 2)[0]]
    y_2 = y[np.where(y == 2)[0]]
    
    indexes = np.where(y==1)[0]
    count = len(indexes)
    
    x1s = []
    y1s = []
    
    for i in range(6):
        xi = []
        yi = []
        
        for j in range(600):
            random = randrange(count)
            index = indexes[random]
        
            xi.append(x[index,:])
            yi.append(y[index,:])
            indexes = np.where(y==1)[0]
            count = len(indexes)
        
        x1s.append(xi)
        y1s.append(yi)
    
    print(x1s)
    print(y1s)
    print('Creat 6 balanced set done')

In [ ]:
create6balancedsets(X_train_copy, y_train_copy)

In [4]:
def resampledDataSet(X_train_copy, y_train_copy):
    
    #Get all the sample on class 0 and 2
    features_0=X_train_copy[np.where(y_train_copy==0)[0]]
    features_2=X_train_copy[np.where(y_train_copy==2)[0]]
    
    #Save their classification / just recreated them would have work too
    y_0=y_train_copy[np.where(y_train_copy==0)[0]]
    y_2=y_train_copy[np.where(y_train_copy==2)[0]]
    
    #Do the same thing for classification 1
    X_label1=X_train_copy[np.where(y_train_copy==1)[0]]
    y_1abel1=y_train_copy[np.where(y_train_copy==1)[0]]
    X_label1, y_1abel1= shuffle(X_label1, y_1abel1, random_state=0)

    #augmented/resample datasets
    #With classification 0
    X_train_augmented=tf.concat([X_train_copy,features_0 ,features_0,features_0,features_0],axis=0) 
    y_train_augmented=tf.concat([y_train_copy.flatten(),tf.zeros(4*features_0.shape[0],dtype="int64")],axis=0)
    
    #With classification 1
    X_train_augmented=tf.concat([X_train_augmented,features_2,features_2,features_2,features_2],axis=0) #along the rows
    y_train_augmented=tf.concat([y_train_augmented,2*tf.ones(4*features_2.shape[0],dtype="int64")],0)

    # Shuffle the data to not biased the model after
    X_train_augmented=tf.random.shuffle(X_train_augmented,seed=8).numpy()
    y_train_augmented=tf.random.shuffle(y_train_augmented,seed=8).numpy().reshape(y_train_augmented.shape[0],1)
  
    #Getting the datasets chunks of label 1
    label1_chunks=[]
    chunks_size=int(X_label1.shape[0]/6)
    for i in range(6):
      label1_chunks.append((X_label1[chunks_size*i:chunks_size*(i+1)],y_1abel1[chunks_size*i:chunks_size*(i+1)]))
  
    #preparing training sets for models
    model_sets=[(X_train_augmented,y_train_augmented)]
    for i in range(6):
      model_set=((np.concatenate((label1_chunks[i][0],features_0,features_2),axis=0),np.concatenate((label1_chunks[i][1],y_0,y_2),axis=0)
   ))
      model_sets.append(shuffle(model_set[0],model_set[1],random_state=1))
    
    return model_sets

In [5]:
sets = resampledDataSet(X_train_copy, y_train_copy)

In [6]:
sets

[(array([[-4.63722289e-01,  7.82043993e-01,  5.81221282e-01, ...,
           1.45173538e+00,  2.01866507e+00,  3.74492466e-01],
         [-4.61253300e-02, -7.55765200e-01,  1.06449440e-01, ...,
           6.73705459e-01,  1.05565190e+00,  3.02166820e-01],
         [-2.65768000e-04,  1.74704447e-01,  8.36888671e-01, ...,
           1.15566886e+00,  7.22084522e-01,  5.68624318e-01],
         ...,
         [-5.97559214e-01, -1.45493582e-01,  8.12333047e-01, ...,
           5.05009234e-01, -1.58621266e-01,  2.18159080e+00],
         [-2.01162115e-01, -7.07909465e-01,  6.51777506e-01, ...,
           3.92371565e-01,  8.39085996e-01,  6.82330060e-02],
         [-6.24226034e-01,  8.96051645e-01,  5.99530518e-01, ...,
           6.38048232e-01,  1.29018784e+00, -3.32590200e-03]]), array([[1],
         [2],
         [0],
         ...,
         [1],
         [0],
         [0]])),
 [array([[-0.08463734, -0.90885973,  0.78172427, ..., -0.43757296,
           2.41921592,  1.73236382],
         [-1.

## Model configuration

Using the tutorial https://medium.com/@canerkilinc/hands-on-tensorflow-2-0-multi-label-classifications-with-mlp-88fc97d6a7e6

In [7]:
# Model 1 : score -> 0.64
# Model 1, under samling randomly -> 0.68
def mymodel1(x, y, batchsize=32, epochs=50):
    #Configure the model (with dropout)
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1000, activation=tf.nn.relu),
        tf.keras.layers.Dense(500, activation=tf.nn.relu),
        tf.keras.layers.Dense(250, activation=tf.nn.relu),
        tf.keras.layers.Dense(125, activation=tf.nn.relu),
        tf.keras.layers.Dense(62, activation=tf.nn.relu),
        tf.keras.layers.Dense(31, activation=tf.nn.relu),
        tf.keras.layers.Dense(15, activation=tf.nn.relu),
        tf.keras.layers.Dense(7, activation=tf.nn.relu),
        tf.keras.layers.Dense(3, activation="softmax")
    ])
    
    #Compile the model
    model.compile(optimizer="adam",
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                 metrics=["sparse_categorical_accuracy"])
    
    #Fit the model
    model.fit(x, y, batch_size=batchsize, epochs=epochs)

    #return the model
    return model

In [8]:
# Model 2 : flatten and droupout layers, score -> 0.61
def mymodel2(x, y, batchsize=32, epochs=50):
    #Configure the model (with dropout)
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1000, activation=tf.nn.relu),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(500, activation=tf.nn.relu),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(250, activation=tf.nn.relu),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(125, activation=tf.nn.relu),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(62, activation=tf.nn.relu),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(31, activation=tf.nn.relu),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(15, activation=tf.nn.relu),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(7, activation=tf.nn.relu),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(3, activation="softmax")
    ])
    
    #Compile the model
    model.compile(optimizer="adam",
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                 metrics=["sparse_categorical_accuracy"])
    
    #Fit the model
    model.fit(x, y, batch_size=batchsize, epochs=epochs)

    #return the model
    return model

In [9]:
# Model 3
def mymodel3(x, y, batchsize=32, epochs=50):
    #Configure the model (with dropout)
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1000, activation=tf.nn.tanh),
        tf.keras.layers.Dense(500, activation=tf.nn.tanh),
        tf.keras.layers.Dense(250, activation=tf.nn.tanh),
        tf.keras.layers.Dense(125, activation=tf.nn.tanh),
        tf.keras.layers.Dense(62, activation=tf.nn.tanh),
        tf.keras.layers.Dense(31, activation=tf.nn.tanh),
        tf.keras.layers.Dense(15, activation=tf.nn.tanh),
        tf.keras.layers.Dense(7, activation=tf.nn.tanh),
        tf.keras.layers.Dense(3, activation="softmax")
    ])
    
    #Compile the model
    model.compile(optimizer="Adadelta" ,
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                 metrics=["sparse_categorical_accuracy"])
    
    #Fit the model
    model.fit(x, y, batch_size=batchsize, epochs=epochs)

    #return the model
    return model

In [10]:
def multiModel(sets, epochs):
    models = []
    for simple_set in sets:
        for epoch in epochs:
            models.append(mymodel1(simple_set[0], simple_set[1], 32, epoch))
            models.append(mymodel2(simple_set[0], simple_set[1], 32, epoch))
            models.append(mymodel3(simple_set[0], simple_set[1], 32, epoch))
            print('train ### neural network#s')
    return models

### Simple model

In [ ]:
M = mymodel1(X_train_copy, y_train_copy)

### Multi-model

In [11]:
Ms = multiModel(sets, [20,30,40,50])

Epoch 1/20
300/300 [==============================] - 2s 8ms/step - loss: 0.7265 - sparse_categorical_accuracy: 0.6762
Epoch 2/20
300/300 [==============================] - 2s 8ms/step - loss: 0.4762 - sparse_categorical_accuracy: 0.8077
Epoch 3/20
300/300 [==============================] - 3s 8ms/step - loss: 0.2549 - sparse_categorical_accuracy: 0.8989
Epoch 4/20
300/300 [==============================] - 2s 8ms/step - loss: 0.1610 - sparse_categorical_accuracy: 0.9428
Epoch 5/20
300/300 [==============================] - 2s 8ms/step - loss: 0.0992 - sparse_categorical_accuracy: 0.9681
Epoch 6/20
300/300 [==============================] - 2s 7ms/step - loss: 0.0610 - sparse_categorical_accuracy: 0.9796
Epoch 7/20
300/300 [==============================] - 2s 7ms/step - loss: 0.0527 - sparse_categorical_accuracy: 0.9835
Epoch 8/20
300/300 [==============================] - 2s 7ms/step - loss: 0.0543 - sparse_categorical_accuracy: 0.9830
Epoch 9/20
300/300 [============================

300/300 [==============================] - 2s 8ms/step - loss: 0.0369 - sparse_categorical_accuracy: 0.9883
Epoch 10/30
300/300 [==============================] - 2s 7ms/step - loss: 0.0239 - sparse_categorical_accuracy: 0.9943
Epoch 11/30
300/300 [==============================] - 2s 7ms/step - loss: 0.0135 - sparse_categorical_accuracy: 0.9959
Epoch 12/30
300/300 [==============================] - 2s 7ms/step - loss: 0.0417 - sparse_categorical_accuracy: 0.9889
Epoch 13/30
300/300 [==============================] - 2s 7ms/step - loss: 0.0288 - sparse_categorical_accuracy: 0.9928
Epoch 14/30
300/300 [==============================] - 2s 7ms/step - loss: 0.0118 - sparse_categorical_accuracy: 0.9967
Epoch 15/30
300/300 [==============================] - 2s 7ms/step - loss: 0.0302 - sparse_categorical_accuracy: 0.9926
Epoch 16/30
300/300 [==============================] - 2s 7ms/step - loss: 0.0297 - sparse_categorical_accuracy: 0.9921
Epoch 17/30
300/300 [==============================]

300/300 [==============================] - 2s 8ms/step - loss: 0.6947 - sparse_categorical_accuracy: 0.7261
Epoch 18/30
300/300 [==============================] - 2s 8ms/step - loss: 0.6892 - sparse_categorical_accuracy: 0.7320
Epoch 19/30
300/300 [==============================] - 2s 8ms/step - loss: 0.6839 - sparse_categorical_accuracy: 0.7365
Epoch 20/30
300/300 [==============================] - 2s 8ms/step - loss: 0.6786 - sparse_categorical_accuracy: 0.7404
Epoch 21/30
300/300 [==============================] - 2s 8ms/step - loss: 0.6734 - sparse_categorical_accuracy: 0.7454
Epoch 22/30
300/300 [==============================] - 2s 8ms/step - loss: 0.6683 - sparse_categorical_accuracy: 0.7505
Epoch 23/30
300/300 [==============================] - 2s 8ms/step - loss: 0.6633 - sparse_categorical_accuracy: 0.7555
Epoch 24/30
300/300 [==============================] - 2s 8ms/step - loss: 0.6583 - sparse_categorical_accuracy: 0.7583
Epoch 25/30
300/300 [==============================]

300/300 [==============================] - 2s 7ms/step - loss: 0.1550 - sparse_categorical_accuracy: 0.9517
Epoch 15/40
300/300 [==============================] - 2s 7ms/step - loss: 0.0949 - sparse_categorical_accuracy: 0.9725
Epoch 16/40
300/300 [==============================] - 2s 8ms/step - loss: 0.1664 - sparse_categorical_accuracy: 0.9467
Epoch 17/40
300/300 [==============================] - 2s 7ms/step - loss: 0.1111 - sparse_categorical_accuracy: 0.9615
Epoch 18/40
300/300 [==============================] - 3s 8ms/step - loss: 0.1069 - sparse_categorical_accuracy: 0.9659
Epoch 19/40
300/300 [==============================] - 2s 8ms/step - loss: 0.1030 - sparse_categorical_accuracy: 0.9658
Epoch 20/40
300/300 [==============================] - 3s 9ms/step - loss: 0.0671 - sparse_categorical_accuracy: 0.9771
Epoch 21/40
300/300 [==============================] - 2s 7ms/step - loss: 0.0965 - sparse_categorical_accuracy: 0.9699
Epoch 22/40
300/300 [==============================]

300/300 [==============================] - 2s 7ms/step - loss: 0.3727 - sparse_categorical_accuracy: 0.8551
Epoch 3/50
300/300 [==============================] - 2s 7ms/step - loss: 0.1892 - sparse_categorical_accuracy: 0.9326
Epoch 4/50
300/300 [==============================] - 2s 7ms/step - loss: 0.1182 - sparse_categorical_accuracy: 0.9603
Epoch 5/50
300/300 [==============================] - 2s 7ms/step - loss: 0.0788 - sparse_categorical_accuracy: 0.9750
Epoch 6/50
300/300 [==============================] - 2s 7ms/step - loss: 0.0666 - sparse_categorical_accuracy: 0.9780
Epoch 7/50
300/300 [==============================] - 2s 7ms/step - loss: 0.0557 - sparse_categorical_accuracy: 0.9821
Epoch 8/50
300/300 [==============================] - 2s 7ms/step - loss: 0.0301 - sparse_categorical_accuracy: 0.9902
Epoch 9/50
300/300 [==============================] - 2s 7ms/step - loss: 0.0193 - sparse_categorical_accuracy: 0.9942
Epoch 10/50
300/300 [==============================] - 2s 7

...

57/57 [==============================] - 0s 7ms/step - loss: 9.5707e-06 - sparse_categorical_accuracy: 1.0000
Epoch 1/30
57/57 [==============================] - 0s 7ms/step - loss: 1.0986 - sparse_categorical_accuracy: 0.3967
Epoch 2/30
57/57 [==============================] - 0s 7ms/step - loss: 1.0193 - sparse_categorical_accuracy: 0.5122
Epoch 3/30
57/57 [==============================] - 0s 7ms/step - loss: 0.9449 - sparse_categorical_accuracy: 0.5372
Epoch 4/30
57/57 [==============================] - 0s 7ms/step - loss: 0.8914 - sparse_categorical_accuracy: 0.5789
Epoch 5/30
57/57 [==============================] - 0s 7ms/step - loss: 0.8618 - sparse_categorical_accuracy: 0.5656
Epoch 6/30
57/57 [==============================] - 0s 8ms/step - loss: 0.7900 - sparse_categorical_accuracy: 0.6111
Epoch 7/30
57/57 [==============================] - 0s 7ms/step - loss: 0.7602 - sparse_categorical_accuracy: 0.6328
Epoch 8/30
57/57 [==============================] - 0s 7ms/step - loss:

57/57 [==============================] - 0s 7ms/step - loss: 0.2967 - sparse_categorical_accuracy: 0.8856
Epoch 10/40
57/57 [==============================] - 0s 7ms/step - loss: 0.2527 - sparse_categorical_accuracy: 0.9050
Epoch 11/40
57/57 [==============================] - 0s 7ms/step - loss: 0.1490 - sparse_categorical_accuracy: 0.9528
Epoch 12/40
57/57 [==============================] - 0s 7ms/step - loss: 0.1250 - sparse_categorical_accuracy: 0.9550
Epoch 13/40
57/57 [==============================] - 0s 7ms/step - loss: 0.1704 - sparse_categorical_accuracy: 0.9361
Epoch 14/40
57/57 [==============================] - 0s 7ms/step - loss: 0.0886 - sparse_categorical_accuracy: 0.9683
Epoch 15/40
57/57 [==============================] - 0s 7ms/step - loss: 0.0611 - sparse_categorical_accuracy: 0.9789
Epoch 16/40
57/57 [==============================] - 0s 7ms/step - loss: 0.0689 - sparse_categorical_accuracy: 0.9783
Epoch 17/40
57/57 [==============================] - 0s 7ms/step - l

57/57 [==============================] - 0s 8ms/step - loss: 0.3626 - sparse_categorical_accuracy: 0.8783
Epoch 39/40
57/57 [==============================] - 0s 7ms/step - loss: 0.2848 - sparse_categorical_accuracy: 0.9183
Epoch 40/40
57/57 [==============================] - 0s 7ms/step - loss: 0.2499 - sparse_categorical_accuracy: 0.9278
Epoch 1/40
57/57 [==============================] - 0s 8ms/step - loss: 1.0160 - sparse_categorical_accuracy: 0.4972
Epoch 2/40
57/57 [==============================] - 0s 8ms/step - loss: 0.9622 - sparse_categorical_accuracy: 0.5594
Epoch 3/40
57/57 [==============================] - 0s 8ms/step - loss: 0.9326 - sparse_categorical_accuracy: 0.5822
Epoch 4/40
57/57 [==============================] - 0s 8ms/step - loss: 0.9141 - sparse_categorical_accuracy: 0.5911
Epoch 5/40
57/57 [==============================] - 0s 8ms/step - loss: 0.9009 - sparse_categorical_accuracy: 0.5983
Epoch 6/40
57/57 [==============================] - 0s 8ms/step - loss: 0

57/57 [==============================] - 0s 7ms/step - loss: 3.7910e-05 - sparse_categorical_accuracy: 1.0000
Epoch 28/50
57/57 [==============================] - 0s 7ms/step - loss: 2.9041e-05 - sparse_categorical_accuracy: 1.0000
Epoch 29/50
57/57 [==============================] - 0s 7ms/step - loss: 2.3037e-05 - sparse_categorical_accuracy: 1.0000
Epoch 30/50
57/57 [==============================] - 0s 7ms/step - loss: 1.8772e-05 - sparse_categorical_accuracy: 1.0000
Epoch 31/50
57/57 [==============================] - 0s 7ms/step - loss: 1.5509e-05 - sparse_categorical_accuracy: 1.0000
Epoch 32/50
57/57 [==============================] - 0s 7ms/step - loss: 1.2977e-05 - sparse_categorical_accuracy: 1.0000
Epoch 33/50
57/57 [==============================] - 0s 7ms/step - loss: 1.0978e-05 - sparse_categorical_accuracy: 1.0000
Epoch 34/50
57/57 [==============================] - 0s 7ms/step - loss: 9.2911e-06 - sparse_categorical_accuracy: 1.0000
Epoch 35/50
57/57 [=================

57/57 [==============================] - 0s 8ms/step - loss: 0.1736 - sparse_categorical_accuracy: 0.9539
Epoch 46/50
57/57 [==============================] - 0s 7ms/step - loss: 0.1466 - sparse_categorical_accuracy: 0.9628
Epoch 47/50
57/57 [==============================] - 0s 7ms/step - loss: 0.1377 - sparse_categorical_accuracy: 0.9672
Epoch 48/50
57/57 [==============================] - 0s 7ms/step - loss: 0.1277 - sparse_categorical_accuracy: 0.9656
Epoch 49/50
57/57 [==============================] - 0s 8ms/step - loss: 0.1181 - sparse_categorical_accuracy: 0.9700
Epoch 50/50
57/57 [==============================] - 0s 7ms/step - loss: 0.1046 - sparse_categorical_accuracy: 0.9772
Epoch 1/50
57/57 [==============================] - 0s 8ms/step - loss: 1.1432 - sparse_categorical_accuracy: 0.3422
Epoch 2/50
57/57 [==============================] - 0s 8ms/step - loss: 1.0689 - sparse_categorical_accuracy: 0.4117
Epoch 3/50
57/57 [==============================] - 0s 8ms/step - loss

## Predictions

### Simple model

In [ ]:
predictions = M.predict(X_test_copy)

In [ ]:
predictions

In [ ]:
predictions=np.argmax(predictions,axis=1)

In [ ]:
predictions

### Multi-model

In [12]:
def multimodelprediction(models, X_test):
    preds_sum = 0
    for model in models :
        preds_sum += model.predict(X_test)
    preds = np.argmax(preds_sum/len(models), axis=1)
    return preds

In [13]:
predictions = multimodelprediction(Ms, X_test_copy)

### Corss validation

In [22]:
def validation(X_train, y_train, models):
    
    preds_sum = 0
    for model in models :
        preds_sum += model.predict(X_train)
    preds = np.argmax(preds_sum/len(models), axis=1)
    
    score=balanced_accuracy_score(y_train,predictions)
    print("Score is : ")

In [23]:
score = validation(X_train_copy, y_train_copy, Ms)

ValueError: Found input variables with inconsistent numbers of samples: [4800, 4100]

## Verification

In [14]:
len(X_test_copy) ==  len(predictions)

True

## Submission

In [15]:
ids = sample.to_numpy()[:,0]
sub = {"id" : ids, "y": predictions}
sub = pd.DataFrame(sub)
print(sub.head())
sub.to_csv("submission_y_test")

    id  y
0  0.0  1
1  1.0  1
2  2.0  0
3  3.0  0
4  4.0  1


## Results

* Model 1, no pre-processing -> 0.64
* Model 1, under-sampling -> 0.68
* Model 2, no pre-processing -> 0.61
* Multi-model, resampling -> 0.724
